###  MRET xlsx to Tidy Data

Take the Trade in goods MRETS (all BOP - EU2013): time series dataset and convert to Tidy Data in CSV.

Fetch and cache the latest published MRETS data as an Excel spreadsheet (single sheet).

In [5]:
import requests
from pathlib import Path
from io import BytesIO
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

inputURL = 'https://www.ons.gov.uk/file?uri=/economy/nationalaccounts/balanceofpayments/datasets/tradeingoodsmretsallbopeu2013timeseriesspreadsheet/current/mret.xlsx'
mretsExcel = BytesIO(session.get(inputURL).content)

ModuleNotFoundError: No module named 'cachecontrol'

Read in the spreadsheet as a table, naming the columns after the CDID (second row).

In [ ]:
import pandas as pd
tab = pd.read_excel(mretsExcel, header=None, na_values=[], keep_default_na=False)
tab.rename(columns=tab.iloc[1], inplace=True)
tab.rename(columns={'CDID': 'Period'}, inplace=True)
tab

Pull out the release date of this dataset, which should be the same for all the time series.

In [ ]:
import datetime
import pytz

release_dates = tab.loc[4][1:].unique()
assert len(release_dates) == 1
release_date = pd.to_datetime(release_dates[0], dayfirst=True).tz_localize('Europe/London').isoformat()
release_date

The observations are in rows 7 on.

In [ ]:
observations = tab[7:].rename(columns={'CDID': 'Period'})
observations.head()

Each CDID corresponds to a unique time-series slice. Unpivot the table so we have one row per observation and drop any rows with no value for the observation.

In [ ]:
observations = pd.melt(observations, id_vars=['Period'], var_name='CDID', value_name='Value')
observations['Value'] = pd.to_numeric(observations['Value'])
observations.dropna(inplace=True)
observations['Value'] = observations['Value'].astype('int64')
observations.reset_index(drop=True, inplace=True)
print(len(observations))
observations.tail(5)

Each CDID is described by a colon separated title, so add these back in to describe the observations.

In [ ]:
observations['Title'] = tab.iloc[0][observations['CDID']].values
observations.head()

The date/time values need to be in a format that can be used to create URIs for British calendar intervals,
see https://github.com/epimorphics/IntervalServer/blob/master/interval-uris.md#british-calendar-intervals

In [ ]:
observations['Period'].unique()

In [ ]:
import re
YEAR_RE = re.compile(r'[0-9]{4}')
YEAR_MONTH_RE = re.compile(r'([0-9]{4})\s+(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)')
YEAR_QUARTER_RE = re.compile(r'([0-9]{4})\s+(Q[1-4])')

# from https://stackoverflow.com/questions/597476/how-to-concisely-cascade-through-multiple-regex-statements-in-python
class Re(object):
  def __init__(self):
    self.last_match = None
  def fullmatch(self,pattern,text):
    self.last_match = re.fullmatch(pattern,text)
    return self.last_match

def time2period(t):
    gre = Re()
    if gre.fullmatch(YEAR_RE, t):
        return f"year/{t}"
    elif gre.fullmatch(YEAR_MONTH_RE, t):
        year, month = gre.last_match.groups()
        month_num = {'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
                     'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}.get(month)
        return f"month/{year}-{month_num}"
    elif gre.fullmatch(YEAR_QUARTER_RE, t):
        year, quarter = gre.last_match.groups()
        return f"quarter/{year}-{quarter}"
    else:
        print(f"no match for {t}")

observations['Period'] = observations['Period'].apply(time2period)
observations

CDID is an arbitrary/opaque 4 letter code registered by ONS and corresponds to a timeseries slice, so each CDID provides the value of a list of dimensions. These codes are also elaborated on in separate spreadsheets currently in Google Drive.

In [ ]:
from IPython.display import display, HTML
cord_sitc_classification_table_url = 'https://drive.google.com/uc?export=download&id=1uJck_DtSgLs0XcEuKDB0swzj1UrWmauj'
cord_sitc_classification_table = BytesIO(session.get(cord_sitc_classification_table_url).content)
cord_sitc_table = pd.read_excel(cord_sitc_classification_table,
                                sheet_name=0, usecols="D:K", index_col=0, skip_footer=1,
                                na_values=[], keep_default_na=False,
                                dtype={'COMMODITY': str, 'AREA': str, 'DIRECTION': str, 'BASIS': str,
                                       'PRICE': str, 'SEASADJ': str, 'PERIOD': str})
display(HTML('<b>CORD SITC Classification table</b>'))
display(cord_sitc_table.head())

In [ ]:
csdb_classification_tables_url = 'https://drive.google.com/uc?export=download&id=1miAzQ6s8om4Ark3BpRk3Y90OAWfWErTb'
csdb_classification_table = BytesIO(session.get(csdb_classification_tables_url).content)
csdb_sheets = pd.read_excel(csdb_classification_table, sheet_name=None, index_col=0,
                            na_values=[], keep_default_na=False,
                            dtype={'COMMODITY': str, 'AREA': str, 'DIRECTION': str, 'BASIS': str,
                                   'PRICE': str, 'SEASADJ': str, 'PERIOD': str, 'PRODUCT': str,
                                   'COUNTRY': str})
for (sheet, df) in csdb_sheets.items():
    display(HTML('<b>' + sheet + '</b>'))
    display(df.head())

The cord_country tab doesn't define product/commodity or price classification. Assume that commodity is 'T' for total and the price is CP for 'current prices'.

__Todo: check this assumption__

In [ ]:
csdb_sheets['cord_country']['COMMODITY'] = 'T'
csdb_sheets['cord_country']['PRICE'] = 'CP'

It looks as though the `cord_sitc_table` is the same as the `cord_sitc` sheet loaded above.

In [ ]:
different_cdids = set(cord_sitc_table.index.values).symmetric_difference(set(csdb_sheets['cord_sitc'].index.values))
display(different_cdids)
cord_sitc_table.sort_index().eq(csdb_sheets['cord_sitc'].sort_index())

Apparently not. Let's use `csdb_sheets` for now. __TODO: need to check whether the CDID period length matches the MRETS period length.__

Next we have another table of CDIDs not listed in the above:

In [ ]:
codelist_url = 'https://drive.google.com/uc?export=download&id=161OtInylx2518gmhRu7UgUYnZZ_x9FQr'
codelist = pd.read_csv(BytesIO(session.get(codelist_url).content), index_col=0,
                       na_values=[], keep_default_na=False,
                       dtype={'AREA': str, 'DIRECTION': str, 'BASIS': str,
                              'PRICE': str, 'SEASADJ': str, 'PERIOD': str},
                       converters={'COMMODITY': lambda x: str(x).strip()})
codelist

This (above) list seems to be missing some values. E.g. looking up `BOQM` on https://www.ons.gov.uk/timeseriestool finds `BOP:Exports:Tons:SA:Crude oil: SITC 333`.

Check that all CDIDs used in MRETS are defined in these tables.

In [ ]:
all_cdids = pd.concat(list(csdb_sheets.values()) + [codelist])
display(all_cdids)
defined_cdids = set(all_cdids.index.values)
remaining = set(observations['CDID'].unique()).difference(defined_cdids)  
assert not remaining, 'Not all CDIDs defined: ' + str(remaining)

Inspect the unique values for these dimensions.

In [ ]:
for col in all_cdids:
    display(HTML('<b>' + col + '</b>'))
    display(all_cdids[col].unique())

__TODO: Still not sure what to make of the titles provided for the CDIDs in the MRETS table.__

__TODO: PERIOD doesn't seem to correspond to usage.__

In [ ]:
bop_series = all_cdids[all_cdids['BASIS'] == 'BOP'].copy()
def area_country(row):
    if pd.isnull(row['AREA']) or row['AREA'] == '':
        if pd.isnull(row['COUNTRY']) or row['COUNTRY'] == '':
            return None
        assert row['COUNTRY'] != ''
        return 'cord/' + row['COUNTRY']
    else:
        assert pd.isnull(row['COUNTRY']) or row['COUNTRY'] == ''
        return 'legacy/' + row['AREA']

bop_series['ONS Partner Geography'] = bop_series.apply(area_country, axis=1)

def product_commodity(row):
    if pd.isnull(row['PRODUCT']) or row['PRODUCT'] == '':
        if pd.isnull(row['COMMODITY']) or row['COMMODITY'] == '':
            return None
        assert not pd.isnull(row['COMMODITY']) and row['COMMODITY'] != ''
        return row['COMMODITY']
    else:
        assert pd.isnull(row['COMMODITY']) or row['COMMODITY'] == ''
        return row['PRODUCT']

bop_series['CORD SITC'] = bop_series.apply(product_commodity, axis=1)

bop_series.drop(columns=['PERIOD', 'AREA', 'COUNTRY', 'PRODUCT', 'COMMODITY', 'BASIS'], inplace=True)
bop_series.rename(columns={'DIRECTION': 'Flow',
                           'PRICE': 'Price Classification',
                           'SEASADJ': 'Seasonal Adjustment'}, inplace=True)
bop_series.replace({'Flow': {'BAL': 'Balance', 'IM': 'Imports', 'EX': 'Exports'}}, inplace=True)
bop_series['Measure Type'] = 'GBP Total'
bop_series['Unit'] = '£ Million'

bop_observations = observations[observations['CDID'].isin(bop_series.index.values)]
bop_observations = bop_observations.merge(bop_series, how = 'left', left_on = 'CDID', right_index=True)
bop_observations.drop(columns=['Title'], inplace=True)
bop_observations.dropna(how='any', inplace=True)
bop_observations

In [ ]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

bop_observations.to_csv(destinationFolder / ('bop_observations.csv'), index = False)

Update dataset metadata

In [ ]:
modified_date = pd.to_datetime('now').tz_localize('Europe/London').isoformat()

from string import Template
with open(Path('metadata') / 'dataset.trig.template', 'r') as metadata_template_file:
    metadata_template = Template(metadata_template_file.read())
    with open(destinationFolder / 'dataset.trig', 'w') as metadata_file:
        metadata_file.write(metadata_template.substitute(issued=release_date, modified=modified_date))